In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
#Run once to pull the GTSRB archive from OneDrive to Gradient and then unzip the contents
#https://arpitapatel.wordpress.com/2017/11/11/how-to-download-files-and-folders-from-onedrive-using-wget/
#wget --no-check-certificate -O GTSRB.zip "https://onedrive.live.com/download?cid=5A94600CB7BF3E5C&resid=5A94600CB7BF3E5C%21119851&authkey=AFAmaa2S07ersB0"
#import zipfile
#with zipfile.ZipFile("GTSRB.zip","r") as zip_ref:
#    zip_ref.extractall("data")
#with zipfile.ZipFile("Test.zip","r") as zip_ref:
#    zip_ref.extractall("data")


In [ ]:
#import pandas as pd
import numpy as np
import os
#import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:
IMAGE_SHAPE = (224, 224)
n_classes = 43

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory='data/Train',
                                                 shuffle=True,
                                                 target_size=IMAGE_SHAPE, 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory='data/Train',
                                                 shuffle=True,
                                                 target_size=IMAGE_SHAPE, 
                                                 subset="validation",
                                                 class_mode='categorical')

In [ ]:
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_image_test_data = image_generator.flow_from_directory(batch_size=32,
                                                 directory='data/Test',
                                                 shuffle=True,
                                                 target_size=IMAGE_SHAPE, 
                                                 subset="training",
                                                 class_mode='categorical')

In [ ]:
pip install sklearn

In [ ]:
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
                class_weight = 'balanced',
                classes = np.unique(train_dataset.classes), 
                y = train_dataset.classes)
class_weights = {i : class_weights[i] for i in range(n_classes)}
class_weights

In [ ]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=True,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Drop FC-1000
    conv_base = Model(inputs=conv_base.inputs, outputs=conv_base.layers[-2].output)
    
    #print(conv_base.summary())
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output

    #top_model = Dropout(0.2)(top_model)
    
    output_layer = Dense(n_classes, activation='softmax')(top_model)
      
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
input_shape = (224, 224, 3)

vgg_model_ft = create_model(input_shape, n_classes)
vgg_model_ft.summary()

In [ ]:
pip install pillow

In [ ]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callback = tf.keras.callbacks.EarlyStopping(restore_best_weights= True, patience=5)
batch_size = 32
history = vgg_model_ft.fit(
    train_dataset,
    batch_size = batch_size,
    steps_per_epoch = train_dataset.samples // batch_size,
    validation_data = validation_dataset, 
    validation_steps = validation_dataset.samples // batch_size,
    verbose = 1,
    epochs = 10, 
    class_weight = class_weights,
          callbacks=[tensorboard_callback])

In [ ]:
history_evaluate = vgg_model_ft.evaluate(test_image_test_data, callbacks=[tensorboard_callback])
history_evaluate